# Important: do this before starting on the project

**Double click here and enter the student numbers for all minigroup members. Don't enter any names.**
1. **First student number**
2. **Second student number**
3. **Third student number.**

Now **[click on this link and read the MATH0011 project instructions.](https://www.ucl.ac.uk/~ucahmto/0011/projectinstructions.html)**

While the mathematics in this project is relatively simple, this project involves writing more (and more complex) Python code than most of the others - though you won't need any Python features beyond what was covered in MATH0011.  It's a good choice for people interested in cryptography or in programming as a career.

# Project 2 - The Vigenère cipher

In this project you will write Python code to automatically decode a classical encryption method called the [Vigenère cipher](https://en.wikipedia.org/wiki/Vigen%C3%A8re_cipher), first invented in the 1500s.  You will begin by using frequency analysis to break the simpler Caesar cipher, which just shifts each letter of the alphabet forward by the same amount, and then move on to use the [Index of Coincidence](https://en.wikipedia.org/wiki/Index_of_coincidence) to break Vigenère.

## Part 1 - frequency analysis and the Caesar cipher

Let $a_0 = a, a_1 = b, a_2 = c, \ldots, a_{25} = z$ be the lower case letters in the English alphabet, and let $a_{26}$ be a space. The **Caesar cipher** with key $\kappa \in \mathbb{Z}$  encrypts a piece of text by changing each letter $a_i$ of the text to $a_{i+\kappa \pmod {27}}$, where $x \pmod {27}$ means the remainder when you divide $x$ by 27. In other words, you move each letter forward $\kappa$ places in the alphabet-plus-space `abcdefghijklmnopqrstuvwxyz␣` (the ␣ character indicates a space), and if you go past space you wrap back round to a.

For example, 
 - the Caesar cipher with key 0 sends `hello` to `hello`,
 - the Caesar cipher with key 1 sends `fuzz␣` to `gv␣␣a`, and
 - the Caesar cipher with key 2 sends `hello␣world` to `jgnnqbyqtnf`.

<!--Usually the keys in the Caesar cipher are letters, not numbers. The letter a corresponds to the key 0 in the description above, the letter b corresponds to the key 1, and so on.  Thus the Caesar cipher with key a would send "hello" to "hello", the Caesar cipher with key b would send "hello" to "ifmmp", and the Caesar cipher with key z would send "hello" to "gdkkp". -->

**Write a function `caesar(text, k)` which encrypts `text` with the key `k` and returns the result.**.

 - you can assume that `text` is a Python string containing only lower case English letters and spaces, and that `k` is an integer
 - your function must return the encrypted text and **not** print anything.

In [0]:
alphabet = 'abcdefghijklmnopqrstuvwxyz '



If you know that a piece of text was encoded using `encodedText = caesar(text, k)`, you can decode it using `caesar(encodedText, -k)`.  Suppose know that `encodedText = caesar(text, k)` but you **do not** know the value of `k`.  To find it, you are going to try all 27 possible shifts and see which one gives you a result that looks like English.

We check whether a text "looks like English" using **frequency analysis**.  If you take a long piece of English text which uses only lower case letters and spaces and count the numbers of spaces, as, bs, cs, and so on, you get a distinctive distribution. The most common character is the space, occuring about 19% of the time, the second most common is e, occuring about 10% of the time, the third most common is t which occurs about 7% of the time, and the least common is z which occurs 0.08% of the time.  In frequency analysis we measure how much a text looks like English by measuring the diffrence between the letter distribution in that text and the known letter distribution in English.

In the cell below I have defined a dictionary `letter_probabilities` such that `english_letter_probabilities[char]` is the approximate probability a randomly chosen letter from a long piece of English text using only lower case letters and spaces is equal to `char`.

In [0]:
alphabet = 'abcdefghijklmnopqrstuvwxyz '
probs = list(map(float, "0.0651738 0.0124248 0.0217339 0.0349835 0.1041442 0.0197881 0.0158610 0.0492888 0.0558094 0.0009033 0.0050529 0.0331490 0.0202124 0.0564513 0.0596302 0.0137645 0.0008606 0.0497563 0.0515760 0.0729357 0.0225134 0.0082903 0.0171272 0.0013692 0.0145984 0.0007836 0.1918182".split()))
letter_probabilities = {alphabet[i]: probs[i] for i in range(nLetters)}

**Write a function `find_caesar_key(encodedText)` which returns the most likely key used to encrypt `encodedText`.**  You can assume that `encodedText` was produced by calling `caesar(text, k)` where `k` is an integer and `text` is a piece of English language text using only lower case letters and spaces, and your function must return `k` and not print anything out.

Your function should work as follows. For each number `k` between 0 and 26,
 - try to decrypt `encodedText` by using `attempt = caesar(encodedText, -k)`
 - work out the number of times each character from the alphabet-plus-space appears in `attempt`,
 - calculate the *score* for `k` as $\operatorname{score}(k) = \sum_{i=0}^{26}(O_i - E_i)^2 / E_i$, where $O_i$ is the number of times the $i$th letter of the alphabet-plus-space appears in `attempt` and $E_i$ is the length of `encodedText` multiplied by the probability that a random letter from an English text is equal to the $i$th letter of the alphabet-plus-space, taken from `english_letter_probabilities`.

Finally, return the value of `k` with the lowest score.

(The *score* is the [chi-squared statistic](https://en.wikipedia.org/wiki/Chi-squared_test) you may have met before).

If you want to split your code into several different functions to make it easier to read and to test, that's fine (in fact it is encouraged!).

You should test your code by calling `find_caesar_key(caesar(text, k))` for various strings `text` and various values of `k` - the output should be `k` so long as the text is long enough.  An example is provided below.

In [0]:
pericles = "the whole earth is the tomb of famous men not only are they commemorated by columns and inscriptions in their own country but in foreign lands there dwells also an unwritten memorial of them graven not on stone but in the hearts of men"
find_caesar_key(caesar(pericles, 4))

## Part 2 - write and test `vigenere(plaintext, key)` and `decrypt_vigenere(ciphertext, key)`

In this part you are going to implement Vigenère encryption and decryption. First we need to define some terms. If we have a method of encrypting text, and we apply it to a string `s` resulting in an encoded string `e`, we call `s` then **plaintext** and `s` then **ciphertext**.


Vigenère encryption uses a word as a key, and each letter in the word is used as the key for a different Caesar cipher, with the letter a corresponding to a shift of 0, the letter b corresponding to a shift of 1, and so on.

If the Vigenère key word `key` has length `n` then 

 - every `n`th letter in the plaintext starting from position 0 is encrypted by a Caesar cipher whose key is `key[0]`
 - every `n`th letter in the plaintext starting from position 1 is encrypted by a Caesar cipher whose key is `key[1]`
 - ...
 - every `n`th letter in the plaintext starting from position n-1 is encrypted by a Caesar cipher whose key is `key[n-1]`
 
For example, if the keyword was `dog` (length 3) and the plaintext was `mathematics` then we would encrypt every third letter starting at the initial m (so m, h, a, c) with a Caesar cipher with shift 3, every third letter starting at the first a (so a, e, t, s) with a Caesar cipher with shift 14, and every third letter starting at the first (t, m, i) with a Caesar cipher with shift 6. The result would be `pozkssdhofg`.

```
shift:      dogdogdogdog
plaintext:  mathematics
ciphertext: pozkssdhofg
```

**Write a function `vigenere(plaintext, key)` which returns the string `plaintext` encrypted using the Vigenère method with `key` as the key.**  You can assume that `plaintext` contains **only** lower case English letters and no other characters.

If we know the key that was used in Vigenère encryption, we can decrypt the ciphertext using the same method we used for encryption except shifting in the opposite direction.  **Write `decrypt_vigenere(ciphertext, key)` so that `decrypt_vigenere(vigenere(plaintext, key), key)` returns `plaintext`.**  You can assume that `plaintext` and `ciphertext` contain **only** lower case English letters and spaces, and no other characters.

## Part 3 - index of coincidence

When we wanted to decrypt a ciphertext created using the Caesar cipher without knowing the key, it was possible to simply try all possible keys and see which gave the best results.   For Vigenère this is not practical: any word (or any string of lower case English characters) of any length could be the key, so we can't try them all.

If we knew the key length $n$, we could regard Vigenère as $n$ different Caesar ciphers and break them each separately using our Caesar methods.  This part is about using the [index of coincidence](https://en.wikipedia.org/wiki/Index_of_coincidence) to estimate the key length from a Vigenère ciphertext.

Given a string `s` of length $N$, let $n_0$ be the number of times the letter a occurs in `s`, $n_1$ the number of times the letter b occurs in `s`, and so on up to $n_{26}$ being the number of spaces.  The index of coincidence for `s` is defined to be

$$\operatorname{ioc}(\texttt{s}) =\frac{\sum_{i=0}^{26}n_i(n_i -1)}{N(N-1)}$$

**Write a function `ioc(s)` which computes the index of coincidence of a Python string `s`.**  You can assume that `s` consists of lower case English letters only.

If you have a ciphertext `c` which resulted from encoding a plaintext consisting of lower case English characters and spaces using a Vigenère cipher with unknown keylength $n$, you can estimate $n$ as

$$\texttt{estimated_key_length}(\texttt{c}) =  \frac{i_{\text{English}} - 1/27}{ \operatorname{ioc}(\texttt{c}) - 1/27}$$

where $i_{\text{English}}$ is the index of coincidence for English text, about 0.079 for lower case characters and spaces only.
It's important to note that this is only an estimate, and will not be exactly correct (indeed, it won't usually be a whole number).

**Write a function `estimated_key_length(c)` computing the quantity above.**

## Part 4 - breaking Vigenère

We are finally ready to create a program that will automatically break the Vigenère cipher.  The method is to use Part 3 to estimate the key length $n$, then to use the function `find_caesar_key` $n$ times to break the $n$ Caesar ciphers that make up a Vigènere cipher with key length $n$.  Since we only have an estimate of the keylength we will have to try several 


**Write a function `break_vigenere(ciphertext)`** which takes a string `ciphertext` as input and then

 - computes the estimated key length, using part 3
 - for each value of n in a small range of values around the estimate, uses `find_caesar_key` to guess a Vigenère key of length n and records the score (as defined in part A) of the result of decrypting the ciphertext with the guessed key
 - prints out the guessed key (as text, not numbers) with the lowest score and the corresponding guessed plaintext

Test your code by

 - creating a string `s` of a few hundred characters of English text (spaces and lower case letters only)
 - choosing a short key `k`
 - creating the ciphertext `c = vigenere(s, k)`
 - calling `break_vigenere(c)`
 
If your code is working correctly, you should get the original text `s` and the secret key `k`.

# Submitting your project

**Make sure you have done all of the following things.**

0. Included **all** minigroup members' student numbers at the top of this notebook.
1. Read through every part of the project to check you have answered all of it.
2. Carefully read and followed all of the [MATH0011 project instructions](https://www.ucl.ac.uk/~ucahmto/0011/projectinstructions.html).
3. Checked that all of your code works correctly.

If you have, you're ready to submit.  One minigroup member only should download the completed notebook (in CoCalc, click the File menu next to the green Save button, then click Download) and submit it on the MATH0011 Moodle.  Please submit **one .ipynb file per minigroup.**